In [1]:
"""
Created on Tue Dec 19 07:16:52 2017

@author: Verena Yacoub
"""

import numpy as np
import cv2

cap = cv2.VideoCapture('solidWhiteRight.mp4')
cv2.namedWindow('frames')
while(cap.isOpened()):
    ret, image = cap.read()

    if ret is True:
        gray = cv2.cvtColor(image,cv2.COLOR_RGB2GRAY) #convert to gray scale
        
             
        kernel_size = 5 #define kernel size for gaussian smoothing
        blur_gray = cv2.GaussianBlur(gray,(kernel_size, kernel_size),0) #performing smoothing or noise reduction
             
        
        low_threshold = 70 # minimum value below which canny filter classify a signal as non edge
        high_threshold = 190# maximum value above which a signal is a sure edge
        edges = cv2.Canny(blur_gray, low_threshold, high_threshold) #apply canny edge detection
             
             
        mask = np.zeros_like(edges)   #create a blank/dark image with the same dimensions as original
        ignore_mask_color = 255 # defining the color of the mask drawn on the dark image (here white)  
        
        
        imshape = image.shape # get the dimensions of the original image
        vertices = np.array([[(0,imshape[0]),(450, 325), (525,325), (890,imshape[0])]], dtype=np.int32) #defining the region of interest
        cv2.fillPoly(mask, vertices, ignore_mask_color) # generate the polygon with the above vertices and fill it in white    
        
            
        masked_edges = cv2.bitwise_and(edges, mask) #returning the image only where mask pixels are nonzero
              
        
        # Define the Hough transform parameters
        rho=2           # distance resolution in pixels of the Hough grid
        theta = np.pi/180   # angular resolution in radians of the Hough grid
        threshold= 15       # minimum number of votes (intersections in Hough grid cell)
        min_line_length= 20       # minimum number of pixels making up a line
        max_line_gap= 20       # maximum gap in pixels between connectable line segments
        
        img= np.copy(image)*0  # again create a blank image with the same original dimensions to draw lines on it    
        lines= cv2.HoughLinesP(masked_edges, rho, theta, threshold, np.array([]), minLineLength=min_line_length, maxLineGap=max_line_gap) # applying probabilistic hough transform
       
        
        thickness=15 # set the thickness of lines to be drawn
        color=[0,0,255] # set the color of lines o be drawn      
      
        lines = lines.reshape(lines.shape[0], lines.shape[2])   # reshape lines to a 2d matrix
        
            
        lines = lines[~np.isnan(lines) & ~np.isinf(lines)] # remove junk from array
        lines.shape = (lines.shape[0]//2,2) # split the x1,y1 and x2,y2 in separate cells
       
            
        right_lines = np.array(list(filter(lambda x: x[0] > (img.shape[1]/2), lines))) #filter points in the right half of the image in a list
        print(right_lines)
        max_right_x, max_right_y = right_lines.max(axis=0) # find max value in the right half
        min_right_x, min_right_y = right_lines.min(axis=0) # find min value in the right half
        
            
        left_lines = np.array(list(filter(lambda x: x[0] < (img.shape[1]/2), lines))) #filter points in the right half of the image in a list
        max_left_x, max_left_y = left_lines.max(axis=0) # find max value in the left half
        min_left_x, min_left_y = left_lines.min(axis=0) # find min value in the right half
        
            
        right_curve = np.poly1d(np.polyfit(right_lines[:,1], right_lines[:,0], 2)) # best fit of the right points
        left_curve  = np.poly1d(np.polyfit(left_lines[:,1], left_lines[:,0], 2)) # best fit of the left points
        
            
        min_y = min(min_left_y, min_right_y) # min vertical value of both image halves so the lines end at the same horizon
        
           
        max_right_x = int(right_curve(img.shape[0])) # find the x value on the fit line at the bottom of image
        min_right_x = int(right_curve(min_right_y)) # find the x value on the fit line at the horizon previously set
        
        min_left_x = int(left_curve(img.shape[0]))  # find the x value on the fit line at the bottom of image
        
        r1 = (min_right_x, min_y) # define upper point of the right fitting line
        r2 = (max_right_x, img.shape[0]) # define bottom point of the right fitting line
        print('Right points r1 and r2,', r1, r2)
        cv2.line(img, r1, r2, color, thickness) # draw right line on the blank image
        
        l1 = (max_left_x, min_y) # define upper point of the left fitting line
        l2 = (min_left_x, img.shape[0]) # define bottom point of the left fitting line
        print('Left points l1 and l2,', l1, l2)
        cv2.line(img, l1, l2, color, thickness) # draw left line on the blank image
                
        lines_edges = cv2.addWeighted(image, 0.8, img, 1, 0) # combine the image with the drawn lines with the original image
        cv2.imshow('frames', lines_edges) # show the combined image
        k= cv2.waitKey(5) & 0xFF
    
        if k==15:
         break

cv2.destroyAllWindows()
cap.release()  




